In [1]:
import sys
from pathlib import Path

import torch
from torch import GradScaler, nn, optim
from torch.utils import data
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.models import segmentation

sys.path.append(str(Path("..").resolve()))
from src.datasets import DATASET_ZOO
from src.learn import CLASS_WEIGHTINGS, CRITERION_ZOO, LR_SCHEDULER_ZOO, OPTIMIZER_ZOO
from src.models import MODEL_ZOO
from src.pipeline import LocalLogger, Trainer, WandbLogger, init_logging
from src.utils.rng import seed
from src.utils.transform import DataAugment, DataTransform

In [ ]:
root = Path(r"../dataset")
entry = DATASET_ZOO["VOC"]
meta = entry.meta
train_transform = DataTransform((500, 500), mask_fill=meta.ignore_index)
train_dataset = entry.construct_train(
    root=root, transforms=train_transform, year="2007"
)
val_dataset = entry.construct_val(root=root, transforms=DataTransform(), year="2007")
train_dataset = data.Subset(train_dataset, range(10))
val_dataset = data.Subset(val_dataset, range(10))
train_loader = DataLoader(train_dataset, batch_size=2, drop_last=True, shuffle=True)
val_loader = DataLoader(val_dataset)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = MODEL_ZOO["deeplabv3_resnet50"](num_classes=meta.num_classes)
weight = CLASS_WEIGHTINGS["log_frequency"](train_dataset, meta.num_classes)
criterion = CRITERION_ZOO["CrossEntropyLoss"](
    ignore_index=meta.ignore_index, weight=weight
)
optimizer = OPTIMIZER_ZOO["SGD"](
    model.parameters(), lr=3e-4, momentum=0.9, weight_decay=5e-4
)

run_folder = Path("../runs/exp1")
checkpoint_file = None
checkpoint_file = Path(r"..\runs\exp1\latest_checkpoint.pth")

In [3]:
trainer = Trainer(
    model,
    train_loader,
    DataAugment(hflip=0.5),
    val_loader,
    DataAugment(),
    criterion,
    optimizer,
    LR_SCHEDULER_ZOO["StepLR"](optimizer, step_size=10, gamma=0.5),
    GradScaler(device),
    device,
    4,
    10,
    meta.num_classes,
    {"aux": 0.5},
    meta.labels,
    meta.colors,
    run_folder,
    checkpoint_steps=5,
    loggers=(LocalLogger(run_folder, meta.labels),),
)

In [ ]:
if checkpoint_file is not None:
    trainer.load_checkpoint(checkpoint_file)
    run_folder.mkdir(parents=True, exist_ok=True)
else:
    run_folder.mkdir(parents=True)

seed(42)
with init_logging(run_folder / "run.log"):
    trainer.train()